# Adding the mixed lubricaiton solver in to slippy
let's add the mixed lubrication solver from abdullah.

First the solver is initialised, this was done with the hertzian solution then the no roughness solution:

The solver basically iterates through these systems:
- solve the reynolds equation
- solve the pressure deflection equation
- update the viscosity and density of the fluid
- check for convergence if not repete the above
- reduce the height change step size if necessary
- check for load balance if not update height repeate above

First we want a lubricant class that allows addition of any number of submodels, these will all be solved at the end of the iteration. This should allow either a common name to be passed and coeffiects or an equation/ funciton. This will mean having set names for the attributes and probably some 'get_attr' magic.

In [1]:
import numpy as np
import slippy.surface as s
from slippy.abcs import _LubricantModelABC, _ReynoldsSolverABC
from slippy.contact import _ModelStep
import typing

# Iterative semi system lubrication step
First we need a step that will hold all the relavant objects and control calling them in the right order

In [ ]:
class SemiSystemStep(_ModelStep):
    def __init__(self, step_name: str, load_z: float, relative_off_set:tuple:None,
                 absolute_off_set: typing.Optional[tuple] = None, max_it=100, 
                 tol=1e-7, interpolation_mode:str = 'nearest', periodic:bool=False,
                 material_options:dict = None, relaxation_factor:float = 0.5)
    def _data_check(self, previous_state:set):
        # check if there is a lubricnt defined for the model
        
        
        pass
    def _solve(self, previous_state:dict, output_file):
        undeformed_gap, surf_1_pts, surf_2_pts = get_gap_from_model(self.model, interferance=0, off_set=off_set,
                                                         mode=opt.interpolation_mode, periodic=opt.periodic)
        if not 'pressure' in previous_state:
            previous_state['pressure'] = np.zeros_like(gap)
        if not 'temperature' in previous_state:
            previous_state['temperature'] = np.zeros_like(gap)
        while load != set_load:
            while not_converged:
                # solve lubricant models
                current_state = self.model.lubricant.solve_pre_models(previous_state)
                # solve reynolds
                current_state = self.reynolds_solver.solve(previous_state, current_state)
                # solve post lubricant models
                current_state = self.model.lubricant.solve_post_models(current_state)
                # solve contact geometry
                disp, deformed_gap = solve_normal_pressure(undeformed_gap, )
        self.solve_sub_models(current_state)
        self.write_ouputs
        return current_state

# Reynolds solver class
Now we want a reynolds solver class, this will solve the reynolds equation it will be held in the step class and called from these

In [ ]:
from inspect import signature

class Lubricant(_LubricantModelABC):
    """Just the basics here, what does a lubricant have to implement?
    
    It must implement a viscosity, density functions that will either return a scalar or a field of the same size as the
    input, the input will contain the ..... pressure and film thickness at each point.
    
    gap functions can be implemented in the steps? 
    
    Parameters
    ----------
    name: str
        The name of the lubricant, used for output and error messages
    reynolds_solver: _ReynoldsSolver
        A reynolds solver which solves the required form of the reynolds equation
    initial_viscosity: {float, np.ndarray}, optional (None)
        Initial values for the viscosity of the lubricant, if this is set and no 
        viscosity model is set the lubricant will be perfectly linear, if this is 
        not set and a lubricant model is set or added later, the values will be initialised
        with the value of the model when all inputs are 0
    inital_density: {float, np.ndarray}, optional (None)
        Initial values for the density of the lubricant, if this is set and no 
        density model is set the lubricant will be incompressable, if this is 
        not set and a lubricant model is set or added later, the values will be initialised
        with the value of the model when all inputs are 0
    viscosity_model: Callable, optional (None)
        A callable which gives pointwise viscosity based on inputs in the current state dict,
        for more information see the add_submodel_function method
    density_model: Callable, optional (None)
        A callable which gives pointwise density based on inputs in the current state dict,
        for more information see the add_submodel_function method
    viscosity_model_inputs: Sequence[str], optional (None)
        The names of the input fields required for the viscosity model
    density_model_inputs: Sequence[str], optional (None)
        The names of the input fields required for the density model
    """
    solver:_ReynoldsSolver = None
    valid_submodel_params = ['viscosities' ,'densities', 'gaps']
    pre_submodels = dict() # keys are the name of the param to be updated, values are tuples, first element is the function, 
    post_submodels = dict()
    initial_viscosity = None
    initial_density = None
    
    def __init__(self, name:str, reynolds_solver:_ReynoldsSolver,
                 initial_viscosity = None, initial_density = None,
                 viscosity_model=None, density_model=None, 
                 viscosity_model_inputs=None, density_model_inputs=None):
        """should be able to set the newtonian properties and solver to be used"""
        self.name = name
        self.reynolds_solver = reynolds_solver
        
        self.initial_viscosity = initial_viscosity
        self.initial_density = initial_density
        
        if viscosity_model is not None:
            self.add_submodel_function('viscosities', viscosity_model, viscosity_model_inputs)
        if density_model is not None:
            self.add_submodel_function('densities', density_model, density_model_inputs)
        
            
    def add_submodel_named(self, model_name, *args, **kwargs, after_reynolds=True):
        """add a named submodel to the lubricaiton solver"""
        
    def add_submodel_function(self, parameter:str, model:typing.Callable, 
                              inputs:typing.Sequence[str], after_reynolds:bool=True):
        """add a sub model for a parameter using a function (pass in the
        current state dict and the this iteration dict)
        viscosity, pressure, gap, 
        
        """
        if not parameter in self.valid_submodel_params:
            raise ValueError(f'{parameter} is not a valid sub model parameter, '
                             f'valid parameters are: {' '.join(self.valid_submodel_params)}')
        if not hasattr(model, '__call__'):
            raise ValueError('Model must be callable')
        
        if type(inputs) is str:
            inputs = (inputs,)
        
        allowed_types = [str, type(None)]
        if any([type(item) not in allowed_types for item in inputs]):
            raise ValueError('Non string inputs found, inputs should be strings relating to '
                             'keys in the current state dictionary, or None for unset inputs')
        
        sig = signature(model)
        params = sig.parameters
        if len(params)!=len(inputs):
            raise ValueError('Number of inputs does not match the number of arguments:'
                             f' {len(inputs)} inputs given, function has {len(params)} arguments')
            
        if after_reynolds:
            self.post_submodels[parameter] = (model, *inputs)
        else:
            self.pre_submodels[parameter] = (model, *inputs)
        
    def data_check(self, current_state):
        if self.solver is None:
            raise ValueError("")
        
    def initialise(self, current_state:dict):
        """Add initial values to the current state dict"""
        pass
    
    def iterate(self, current_state:dict):
        """Iterates the lubricant model solving all the submodels and the reynolds solver
        
        Parameters
        """
        while True:
            #solve pre models
            for param_name, (model, *args_names) in self.pre_submodels.items():
                args = [current_state[arg_name] for arg_name in arg_names]
                current_state[param_name] = model(*args)
            
            #solve reynolds equaiton
            current_state = self.reynolds_solver.solve(current_state)
            
            # solve post models
            for param_name, (model, *args_names) in self.post_submodels.items():
                args = [current_state[arg_name] for arg_name in arg_names]
                current_state[param_name] = model(*args)
            
            current_state = yield current_state

Next we will need to implement the reynolds solver as a class

In [ ]:
import abc

class _ReynoldsSolver(abc.ABC):
    """Abstract base class for reynolds solvers this should take only solver specific inputs
    
    To implement your own reynolds solver subclass this ABC, you 
    """
    _subclass_registry = []
    
    def __init__(self):
        pass
    
    @classmethod
    def __init_subclass__(cls, is_abstract=False, **kwargs):
        super().__init_subclass__(**kwargs)
        if not is_abstract:
            _ReynoldsSolver._subclass_registry.append(cls)
        
    @abc.abstractmethod
    def solve(self, previous_state:dict)->dict:
        pass
    
    @abc.abstractmethod
    def dimentionalise(self, )
    
    @abc.abstractmethod
    def __repr__(self):
        pass

In [ ]:
class UnifiedSolver(_ReynoldsSolver):
    """This class does the solving of the reynolds equations"""
    def __init__(self, relaxation_factor, dimention_less=True):
        self.relaxation_factor = relaxation_factor
        
    def solve(self, current_state):
        # find the reynolds coefficent (xi) EPS in fortran code
        
        # find first 

Next we will extend the steps class to make it model step and avalible to be solved with the same api as the current steps. This step will just implement the global optimisation 

In [5]:
my_dict = {'a': [1,2,3]}
a, (f, *p) = my_dict.items()

ValueError: not enough values to unpack (expected 2, got 1)

In [3]:
my_dict.items()

dict_items([('a', [1, 2, 3])])

In [6]:
for a, (f, *p) in my_dict.items():
    print(a)

a


In [7]:
f

1

In [8]:
p

[2, 3]

In [9]:
md= {a:[f,*p]}

In [10]:
md

{'a': [1, 2, 3]}